Q1. Derive points table for icc world cup.

In [0]:

columns = ['Team_1', 'Team_2', 'Winner']
data = [('India','SL','India'), ('SL','Aus','Aus'), ('SA','Eng','Eng'), ('Eng','NZ','NZ'), ('Aus','India','India') ]
df = spark.createDataFrame(data, columns)
from pyspark.sql.functions import when, count, sum, col
df_1 = df.select(col('Team_1').alias('Team'), when(df.Team_1==df.Winner, 1).otherwise(0).alias('win_flag'))
df_2 = df.select(col('Team_2').alias('Team'), when(df.Team_2==df.Winner, 1).otherwise(0).alias('win_flag'))
df_final = df_1.union(df_2).groupBy('Team').agg(count('Team').alias('Matches_played'), sum('win_flag').alias('wins')).withColumn('loses', col("matches_played") - col("wins"))  
df_final.show()

+-----+--------------+----+-----+
| Team|Matches_played|wins|loses|
+-----+--------------+----+-----+
|India|             2|   2|    0|
|   SL|             2|   0|    2|
|   SA|             1|   0|    1|
|  Eng|             2|   1|    1|
|  Aus|             2|   1|    1|
|   NZ|             1|   1|    0|
+-----+--------------+----+-----+



Q2. Find the number of new and repeat customers

In [0]:
\
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, to_date, min
schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("order_date", StringType(), True),
    StructField("order_amount", IntegerType(), True)
])
data = [
    (1, 100, '2022-01-01', 2000),
    (2, 200, '2022-01-01', 2500),
    (3, 300, '2022-01-01', 2100),
    (4, 100, '2022-01-02', 2000),
    (5, 400, '2022-01-02', 2200),
    (6, 500, '2022-01-02', 2700),
    (7, 100, '2022-01-03', 3000),
    (8, 400, '2022-01-03', 1000),
    (9, 600, '2022-01-03', 3000)
]
#method1
customer_orders_df = spark.createDataFrame(data, schema)
Total_customers_df = customer_orders_df.groupBy('order_date').agg(count('*').alias('total_customers'))
new_customers_df = customer_orders_df.alias('o1').join(customer_orders_df.alias('o2'), (col('o1.customer_id')==col('o2.customer_id'))&(col('o1.order_date')>col('o2.order_date')), 'left' ).filter(col('o2.order_id').isNull()).select(col('o1.order_date').alias('order_date'))
new_customers_df = new_customers_df.groupBy('order_date').agg(count('*').alias('new_customers'))
final_df = Total_customers_df.join(new_customers_df, 'order_date', 'fullouter')
final_df = final_df.withColumn('repeat_customers', col('total_customers')-col('new_customers'))
final_df.show()

+----------+---------------+-------------+----------------+
|order_date|total_customers|new_customers|repeat_customers|
+----------+---------------+-------------+----------------+
|2022-01-01|              3|            3|               0|
|2022-01-02|              3|            2|               1|
|2022-01-03|              3|            1|               2|
+----------+---------------+-------------+----------------+



In [0]:
#method2
from pyspark.sql.functions import col, to_date, min, when
customer_orders_df = customer_orders_df.withColumn('order_date', to_date(col('order_date'), 'yyyy-MM-dd'))

first_orders_df = customer_orders_df.groupBy('customer_id').agg(min('order_date').alias('first_order_date'))
new_df = customer_orders_df.join(first_orders_df, 'customer_id', 'inner').withColumn('new_customers_flg', when(col('order_date')==col('first_order_date'),1).otherwise(0)).withColumn('repeat_customers_flg', when(col('order_date')!=col('first_order_date'),1).otherwise(0))
ans_df = new_df. groupBy('order_date').agg(sum('new_customers_flg').alias('new_customers'), sum('repeat_customers_flg').alias('repeat_customers'))
ans_df.show()

+----------+-------------+----------------+
|order_date|new_customers|repeat_customers|
+----------+-------------+----------------+
|2022-01-01|            3|               0|
|2022-01-02|            2|               1|
|2022-01-03|            1|               2|
+----------+-------------+----------------+



Q3. scenario based question
-- output should be in the below format
-- name, total_visits, most visisted floor, resources used

In [0]:

entries_data = [
    ('A', 'Bangalore', 'A@gmail.com', 1, 'CPU'),
    ('A', 'Bangalore', 'A1@gmail.com', 1, 'CPU'),
    ('A', 'Bangalore', 'A2@gmail.com', 2, 'DESKTOP'),
    ('B', 'Bangalore', 'B@gmail.com', 2, 'DESKTOP'),
    ('B', 'Bangalore', 'B1@gmail.com', 2, 'DESKTOP'),
    ('B', 'Bangalore', 'B2@gmail.com', 1, 'MONITOR')
]
from pyspark.sql.functions import col, count, collect_set, row_number
entries_columns = ["name", "address", "email", "floor", "resources"]
entries_df = spark.createDataFrame(entries_data, entries_columns)
floors_visits_df = entries_df.groupBy('name', 'floor').agg(count('*').alias('visit_times')).withColumn('rn', row_number().over(Window.partitionBy('name').orderBy(col('visit_times').desc())))
total_visits_df = entries_df.groupBy('name').agg(count('name').alias('total_visits'), collect_set('resources').alias('resources_used'))
final_df = total_visits_df.join(floors_visits_df, 'name', 'inner').filter(col('rn')==1).select('name','total_visits', col('floor').alias('most_visited_floor'),'resources_used')
final_df.show()


+----+------------+------------------+------------------+
|name|total_visits|most_visited_floor|    resources_used|
+----+------------+------------------+------------------+
|   A|           3|                 1|    [DESKTOP, CPU]|
|   B|           3|                 2|[DESKTOP, MONITOR]|
+----+------------+------------------+------------------+



Q4.write pyspark code for finding Date for nth occurence of day in future from given date

In [0]:

from pyspark.sql.functions import col, lit, expr, date_add
from datetime import datetime, timedelta
data = [("2023-12-21", 3)]  
columns = ["given_date", "nth_occurrence"]
input_df = spark.createDataFrame(data, columns)

def get_nth_sunday(given_date: str, nth: int) -> str:
    start_date = datetime.strptime(given_date, "%Y-%m-%d")
    days_to_next_sunday = (6 - start_date.weekday()) % 7  
    first_sunday = start_date + timedelta(days=days_to_next_sunday)
    nth_sunday = first_sunday + timedelta(weeks=(nth - 1))
    return nth_sunday.strftime("%Y-%m-%d")
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

nth_sunday_udf = udf(get_nth_sunday, StringType())
result_df = input_df.withColumn(
    "nth_sunday", nth_sunday_udf(col("given_date"), col("nth_occurrence"))
)

result_df.show()


+----------+--------------+----------+
|given_date|nth_occurrence|nth_sunday|
+----------+--------------+----------+
|2023-12-21|             3|2024-01-07|
+----------+--------------+----------+




Q6. write a  query to find person id, name, number of friends, sum of marks of the person who has friends with total Score
-- greater than 100

In [0]:

friend_data = [
    (1, 2),
    (1, 3),
    (2, 1),
    (2, 3),
    (3, 5),
    (4, 2),
    (4, 3),
    (4, 5)
]
person_data = [
    (1, "Alice", 88),
    (2, "Bob", 11),
    (3, "Devis", 27),
    (4, "Tara", 45),
    (5, "John", 63)
]
from pyspark.sql.functions import col, min, count, sum
friend_schema = ["pid", "fid"]
person_schema = ["PersonID", "Name", "Score"]
friend_df = spark.createDataFrame(friend_data, schema=friend_schema)
person_df = spark.createDataFrame(person_data, schema=person_schema)
df = friend_df.join(person_df, friend_df.fid == person_df.PersonID, 'inner').select('pid', 'fid', col('score').alias('f_score'))
df = df.join(person_df, df.pid == person_df.PersonID, 'inner').select('pid', 'fid', 'f_score', 'Name', col('Score').alias('P_score'))
final_df = df.groupBy('pid').agg(min('Name').alias('Name'), count('fid').alias('num_friends'), sum('f_score').alias('sf_score'), min('p_score').alias('person_score')).filter(col('sf_score')>100).drop(col('sf_score'))
final_df.show()



+---+----+-----------+------------+
|pid|Name|num_friends|person_score|
+---+----+-----------+------------+
|  2| Bob|          2|          11|
|  4|Tara|          3|          45|
+---+----+-----------+------------+




Q7.Write a SQL query to find the cancellation rate of requests with unbanned users 
(both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03".
Round Cancellation Rate to two decimal points.
I The cancellation rate is computed by dividing the number of canceled (by client or driver)
requests  with unbanned users by the total number of requests with unbanned users on that day.

In [0]:

trips_data = [
    (1, 1, 10, 1, 'completed', '2013-10-01'),
    (2, 2, 11, 1, 'cancelled_by_driver', '2013-10-01'),
    (3, 3, 12, 6, 'completed', '2013-10-01'),
    (4, 4, 13, 6, 'cancelled_by_client', '2013-10-01'),
    (5, 1, 10, 1, 'completed', '2013-10-02'),
    (6, 2, 11, 6, 'completed', '2013-10-02'),
    (7, 3, 12, 6, 'completed', '2013-10-02'),
    (8, 2, 12, 12, 'completed', '2013-10-03'),
    (9, 3, 10, 12, 'completed', '2013-10-03'),
    (10, 4, 13, 12, 'cancelled_by_driver', '2013-10-03')
]
trips_columns = ["id", "client_id", "driver_id", "city_id", "status", "request_at"]
trips_df = spark.createDataFrame(trips_data, trips_columns)
users_data = [
    (1, 'No', 'client'),
    (2, 'Yes', 'client'),
    (3, 'No', 'client'),
    (4, 'No', 'client'),
    (10, 'No', 'driver'),
    (11, 'No', 'driver'),
    (12, 'No', 'driver'),
    (13, 'No', 'driver')
]
from pyspark.sql.functions import when
users_columns = ["users_id", "banned", "role"]
users_df = spark.createDataFrame(users_data, users_columns)
uc_df = trips_df.join(users_df, (trips_df.client_id == users_df.users_id) & (users_df.banned != 'Yes'), 'inner').select("id", "client_id", "driver_id", "city_id", "status", "request_at")
ucd_df = uc_df.join(users_df, (trips_df.driver_id == users_df.users_id) & (users_df.banned != 'Yes'), 'inner').select("id", "client_id", "driver_id", "city_id", "status", "request_at")
ucd_df = ucd_df.withColumn('flg', when(col('status') != 'completed', 1).otherwise(0))
cancellation_rates_df = ucd_df.groupBy('request_at').agg( (sum('flg')*100/count('*')).alias('cancelled_trips_rates'))
cancellation_rates_df.show()


+----------+---------------------+
|request_at|cancelled_trips_rates|
+----------+---------------------+
|2013-10-03|                 50.0|
|2013-10-01|   33.333333333333336|
|2013-10-02|                  0.0|
+----------+---------------------+



Q8. Write an SQL query to find the winner in each group.
The winner in each group is the player who scored the maximum total points within the group.
In the case of a tie, the lowest player_id wins.

In [0]:

players_data = [
    (15, 1),
    (25, 1),
    (30, 1),
    (45, 1),
    (10, 2),
    (35, 2),
    (50, 2),
    (20, 3),
    (40, 3)
]
players_columns = ["player_id", "group_id"]
players_df = spark.createDataFrame(players_data, players_columns)
matches_data = [
    (1, 15, 45, 3, 0),
    (2, 30, 25, 1, 2),
    (3, 30, 15, 2, 0),
    (4, 40, 20, 5, 2),
    (5, 35, 50, 1, 1)
]
from pyspark.sql.functions import col, sum, row_number
from pyspark.sql.window import Window
matches_columns = ["match_id", "first_player", "second_player", "first_score", "second_score"]
matches_df = spark.createDataFrame(matches_data, matches_columns)
first_player_df = matches_df.select(col('first_player').alias('player_id'), col('first_score').alias('score'))
second_player_df = matches_df.select(col('second_player').alias('player_id'), col('second_score').alias('score'))
player_score_df = first_player_df.union(second_player_df).groupBy('player_id').agg(sum('score').alias('score'))
final_df = players_df.join(player_score_df, 'player_id', 'inner')
window_spec = Window.partitionBy('group_id').orderBy(col('score').desc(), 'player_id')
final_df = final_df.withColumn('rn', row_number().over(window_spec)).filter(col('rn')==1)
final_df.show()

+---------+--------+-----+---+
|player_id|group_id|score| rn|
+---------+--------+-----+---+
|       15|       1|    3|  1|
|       35|       2|    1|  1|
|       40|       3|    5|  1|
+---------+--------+-----+---+



Q9. MARKET ANALYSIS: Write an SQL query to find for each seller, whether the brand of the
second item (by date) they sold is their favor.
If a seller sold less than two items, report the answer for that seller as no.
o/p
seller id   2nd_item_fav_brand
1   yes/no
2   yes/no
*/


In [0]:

users_data = [
    (1, '2019-01-01', 'Lenovo'),
    (2, '2019-02-09', 'Samsung'),
    (3, '2019-01-19', 'LG'),
    (4, '2019-05-21', 'HP')
]
users_columns = ["user_id", "join_date", "favorite_brand"]
users_df = spark.createDataFrame(users_data, users_columns)
items_data = [
    (1, 'Samsung'),
    (2, 'Lenovo'),
    (3, 'LG'),
    (4, 'HP')
]
items_columns = ["item_id", "item_brand"]
items_df = spark.createDataFrame(items_data, items_columns)
orders_data = [
    (1, '2019-08-01', 4, 1, 2),
    (2, '2019-08-02', 2, 1, 3),
    (3, '2019-08-03', 3, 2, 3),
    (4, '2019-08-04', 1, 4, 2),
    (5, '2019-08-04', 1, 3, 4),
    (6, '2019-08-05', 2, 2, 4)
]
from pyspark.sql.window import Window
from pyspark.sql.functions import nth_value
orders_columns = ["order_id", "order_date", "item_id", "buyer_id", "seller_id"]
orders_df = spark.createDataFrame(orders_data, orders_columns)
df = orders_df.join(items_df, 'item_id', 'inner').join(users_df, orders_df.seller_id == users_df.user_id, 'right').select('order_date', 'user_id', 'item_brand', 'favorite_brand' )
window_spec = Window.partitionBy('user_id').orderBy('order_date').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df = df.withColumn('2nd_sold_item', nth_value('item_brand', 2).over(window_spec))
df = df.groupBy('user_id').agg(min('favorite_brand').alias('fav_brand'), min('2nd_sold_item').alias('2nd_sold_item'))
final_df = df.select('user_id', when(col('fav_brand')==col('2nd_sold_item'), 'Yes').otherwise('No').alias('sold_2nd_fav_item'))
final_df.show()

+-------+-----------------+
|user_id|sold_2nd_fav_item|
+-------+-----------------+
|      1|               No|
|      2|              Yes|
|      3|              Yes|
|      4|               No|
+-------+-----------------+



Q10. group the below data into timeframe for continuous same events

In [0]:

tasks_data = [
    ('2019-01-01', 'success'),
    ('2019-01-02', 'success'),
    ('2019-01-03', 'success'),
    ('2019-01-04', 'fail'),
    ('2019-01-05', 'fail'),
    ('2019-01-06', 'success')
]
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, when, to_date, min, max
tasks_columns = ["date_value", "state"]
tasks_df = spark.createDataFrame(tasks_data, tasks_columns)
tasks_df = tasks_df.withColumn('date_value', to_date(col('date_value'), 'yyyy-MM-dd'))
window_spec = Window.orderBy('date_value')
tasks_df = tasks_df.withColumn('prev_state', lag(col('state'),1,'success' ).over(window_spec)).withColumn('flg', when(col('state')==col('prev_state'),0).otherwise(1) )
tasks_df = tasks_df.withColumn('grp', sum('flg').over(window_spec))
tasks_df = tasks_df.groupBy('grp').agg(min('date_value').alias('start_date'), max('date_value').alias('end_date'), min('state').alias('state'))
tasks_df.show()


+---+----------+----------+-------+
|grp|start_date|  end_date|  state|
+---+----------+----------+-------+
|  0|2019-01-01|2019-01-03|success|
|  1|2019-01-04|2019-01-05|   fail|
|  2|2019-01-06|2019-01-06|success|
+---+----------+----------+-------+

